## Case Study 3 (CS3) - Sill Complex, Northwestern Australia
### Model D - Built using our adaption of the ODSIM (Henrion et al. 2008,2010) and structural frames (Grose et al. 2021).
- Considers a sample of approximately a 0.1% of the original dataset (Koepping et al. 2021). These points were classified depending on their location as top, base and lateral contacts. 
- Compared to Model C, Model D considers the emplacement model proposed by Koepping et al. (2021). The emplacement model is simplified as two sill propagating to the north and south of a central fault (interpreted as the feeder). Transgresive outer sills are controled by marginal faults.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Loop library
from LoopStructural import GeologicalModel
from LoopStructural.visualisation import LavaVuModelViewer 

In [ ]:
lower_extent = [189000, 7775000, -12000]
upper_extent = [196350, 7790050, -8500]

#### 1. Load data

##### Input DataFrame description
> feature_name = name of the geological feature to be modelled. Faults 3 and 6 are the marginal faults that facilitated the emplacement of the inward-dipping outer sills

> (X, Y, Z) = data points location

> coord = structural frame coordinate

> val = values of the scalar field for interpolation of geological features

> (gx, gy, gz) = gradients of structural frame scalar fields

> intrusion_contact_type = roof/top or floor/base

> intrusion_side = TRUE if lateral contact, blank if not

> intrusion_anisotropy = name of geological feature in contact with intrusion (i.e. fault or stratigraphic unit)

In [ ]:
model_data = pd.read_csv('S3 Sill - Model D.csv')
northern_sill_data = model_data[model_data['feature_name'] == 'S3 northern sill'].copy()
southern_sill_data = model_data[model_data['feature_name'] == 'S3 southern sill'].copy()

model_data.head()

#### 2. Define conceptual model function to constraint lateral and vertical extent

These conceptual models are used to constraint the simulation of the intrusion lateral and vertical extent.

In [ ]:
def ellipse_function(
    lateral_contact_data = pd.DataFrame() , model = False, minP=None, maxP=None, minS=None, maxS=None
):
    
    if lateral_contact_data.empty:
        return model, minP, maxP, minS, maxS
    else:
        if minP == None:
            minP = lateral_contact_data["coord1"].min()
        if maxP == None:
            maxP = lateral_contact_data["coord1"].max()
        if minS == None:
            minS = lateral_contact_data["coord2"].abs().min()
        if maxS == None:
            maxS = lateral_contact_data["coord2"].max()

        a = (maxP - minP) / 2
        b = (maxS - minS) / 2

        po = minP + (maxP - minP) / 2

        p_locations = lateral_contact_data.loc[:, "coord1"].copy().to_numpy()

        s = np.zeros([len(p_locations), 2])

        s[np.logical_and(p_locations>minP, p_locations<maxP),0] =  b * np.sqrt(1 - np.power((p_locations[np.logical_and(p_locations>minP, p_locations<maxP)] - po) / a, 2)) 
        s[np.logical_and(p_locations>minP, p_locations<maxP),1] =  -b * np.sqrt(1 - np.power((p_locations[np.logical_and(p_locations>minP, p_locations<maxP)] - po) / a, 2)) 

        return s


def constant_function(
    othercontact_data = pd.DataFrame(),
    mean_growth=None, 
    minP=None, 
    maxP=None, 
    minS=None, 
    maxS=None, 
    vertex=None):
    
    if othercontact_data.empty:
        return mean_growth
    
    if mean_growth == None:
        mean_growth = othercontact_data.loc[:,'coord0'].mean()
        
    data_ps = np.array([othercontact_data.loc[:,'coord1'], othercontact_data.loc[:,'coord2']]).T
    
    conceptual_growth = np.ones([len(data_ps),2]) * mean_growth
    
    return conceptual_growth

#### 3. Create geological model

In [ ]:
#functions for visualization of faults

def fault3_extent(points):
    # points: array ([x,y,z])
    # maximum y value is scaled to model
    mask = np.zeros(len(points)).astype(bool)
    mask[points[:,1]< 9424.15] = 1
    
    return mask

def fault7_extent(points):
    # points: array ([x,y,z])
    # minimum and maximum x values are scaled to model
    mask1 = np.zeros(len(points)).astype(bool)
    mask2 = np.zeros(len(points)).astype(bool)
    mask1[points[:,0]> 3400.75] = 1
    mask2[points[:,0]< 6924.82] = 1
    return mask1 * mask2

In [ ]:
# # Create Geological Model of pre-intrusion geology

model = GeologicalModel(lower_extent,upper_extent)
displacement = 0
# model.nsteps = [50,50,150]
model.data = model_data

fault3 = model.create_and_add_fault('fault_3',displacement,nelements=2000,steps=1,interpolatortype='FDI',buffer=0.3)
fault6 = model.create_and_add_fault('fault_6',displacement,nelements=2000,steps=1,interpolatortype='FDI',buffer=0.3)
fault7 = model.create_and_add_fault('fault_7',displacement,nelements=2000,steps=1,interpolatortype='FDI',buffer=0.3)

conformable_feature = model.create_and_add_foliation('stratigraphy', nelements = 3000, solver = 'lu', interpolatortype = 'FDI')

# Visualization of pre-intrusion geology

viewer_geo = LavaVuModelViewer(model, background='white')

# add stratigraphy
viewer_geo.add_isosurface(conformable_feature, isovalue = 0, colour = 'grey')

# add faults
viewer_geo.add_isosurface(fault3[0], isovalue = 0, colour = 'blue', region = fault3_extent)
viewer_geo.add_isosurface(fault6[0], isovalue = 0, colour = 'deepblue')
viewer_geo.add_isosurface(fault7[0], isovalue = 0, colour = 'lightblue',region = fault7_extent)

viewer_geo.add_points(northern_sill_data.loc[:,['X','Y','Z']].to_numpy(), name ='northern sill contact data', colour = 'black', pointsize = 6)
viewer_geo.add_points(southern_sill_data.loc[:,['X','Y','Z']].to_numpy(), name ='southern sill contact data', colour = 'black', pointsize = 6)

viewer_geo.interactive()

In [ ]:
stratigraphic_column = {}
stratigraphic_column['stratigraphy'] = {}
stratigraphic_column['stratigraphy']['Unit_0'] = {'min':-100,'max':5,'id':0}
stratigraphic_column['stratigraphy']['Unit_1'] = {'min':5,'max':40,'id':1}
model.set_stratigraphic_column(stratigraphic_column)

In [ ]:
def faults_3_6_footwall_fx(xyz):
    fault_3_fw = fault3[0].evaluate_value(xyz)
    ftwall = np.logical_not(fault_3_fw<0)
    return ftwall

In [ ]:
from datetime import datetime
datetime.now().isoformat(timespec='seconds')   

In [ ]:
# # Add intrusions

intrusion_frame_parameters = {'contact' :'floor', 
                              'contact_anisotropies' : [conformable_feature],
                              'delta_c' : [[1,0.1]],
                              'delta_f' : [0.005],
                              'g_w':1e-3
                             }

intrusion_steps = {}
intrusion_steps['step1'] = {'structure' : fault6, 
                            'unit_from' : 'Unit_0', 'series_from': conformable_feature,
                            'unit_to' : 'Unit_1','series_to': conformable_feature}

intrusion_frame_parameters['intrusion_steps'] = intrusion_steps

print(datetime.now().isoformat(timespec='seconds'))


Northern_Sill = model.create_and_add_intrusion('S3 northern sill', intrusion_frame_name = 'S3 northern sill frame',
                                               intrusion_lateral_extent_model = ellipse_function,
                                               intrusion_vertical_extent_model = constant_function,
                                               intrusion_frame_parameters = intrusion_frame_parameters,
                                              interpolatortype = 'FDI')

print(datetime.now().isoformat(timespec='seconds'))


intrusion_frame_parameters = {'type' : 'interpolated' , 'contact' :'floor', 
                              'contact_anisotropies' : [conformable_feature],
                              'delta_c' : [[0.5,0.05], [0,0.05]],
                              'delta_f' : [1, .1],
                              # 'g_w':1e-4
                              
                             }

intrusion_steps = {}
intrusion_steps['step1'] = {'structure' : fault6, 
                            'unit_from' : 'Unit_0', 'series_from': conformable_feature,
                            'unit_to' : 'Unit_1','series_to': conformable_feature,
                            'region': faults_3_6_footwall_fx,
                           }
intrusion_steps['step2'] = {'structure' : fault3, 
                            'unit_from' : 'Unit_0', 'series_from': conformable_feature,
                            'unit_to' : 'Unit_1','series_to': conformable_feature}

intrusion_frame_parameters['intrusion_steps'] = intrusion_steps

print(datetime.now().isoformat(timespec='seconds'))

Southern_Sill = model.create_and_add_intrusion('S3 southern sill', intrusion_frame_name = 'S3 southern sill frame',
                                               intrusion_lateral_extent_model = ellipse_function,
                                               intrusion_vertical_extent_model = constant_function,
                                               intrusion_frame_parameters = intrusion_frame_parameters,
                                               interpolatortype = 'FDI')

print(datetime.now().isoformat(timespec='seconds'))

In [ ]:
roof_data = southern_sill_data[southern_sill_data['intrusion_contact_type'] == 'roof'].copy()
floor_data = southern_sill_data[southern_sill_data['intrusion_contact_type'] == 'floor'].copy()

In [ ]:
intrusion_frame =Northern_Sill.intrusion_frame

viewer = LavaVuModelViewer(model, background='white')
# viewer.nsteps = [25,25,25]

viewer.add_isosurface(fault6[0], isovalue = 0, colour = 'deepblue')
viewer.add_isosurface(fault3[0], isovalue = 0, colour = 'deepblue')
# viewer.add_points(roof_data.loc[:,['X','Y','Z']].to_numpy(), name = 'roof sill data', pointsize = 5, colour = 'green')
# viewer.add_points(floor_data.loc[:,['X','Y','Z']].to_numpy(), name = 'floor sill data', pointsize = 5, colour = 'blue')

# # # add stratigraphy
viewer.add_isosurface(intrusion_frame[0], isovalue = 0, colour = 'red')
# viewer.add_isosurface(intrusion_frame[1], isovalue = 0, colour = 'blue')
# viewer.add_isosurface(intrusion_frame[2], isovalue = 0, colour = 'green')
viewer.add_data(intrusion_frame[0])
# 
# viewer.add_isosurface(conformable_feature, isovalue = 0, colour = 'grey')


viewer.ymin = 0
viewer.xmin = 0
viewer.ymax = 1
viewer.xmax = 1
viewer.interactive()

In [ ]:
nn = 100 # improve model resolution by changing this value
model.nsteps = [nn,nn,nn*5]

In [ ]:
datetime.now().isoformat(timespec='seconds')   

In [ ]:
viewer = LavaVuModelViewer(model, background='white')

viewer.add_points(northern_sill_data.loc[:,['X','Y','Z']].to_numpy(), name = 'Northern sill data', pointsize = 5)
viewer.add_points(southern_sill_data.loc[:,['X','Y','Z']].to_numpy(), name = 'Southern sill data', pointsize = 5)

# viewer.add_points(roof_data.loc[:,['X','Y','Z']].to_numpy(), name = 'roof sill data', pointsize = 5, colour = 'green')
# viewer.add_points(floor_data.loc[:,['X','Y','Z']].to_numpy(), name = 'floor sill data', pointsize = 5, colour = 'blue')

# # add stratigraphy
viewer.add_isosurface(conformable_feature, isovalue = 0, colour = 'grey')

# # # add faults
viewer.add_isosurface(fault3[0], isovalue = 0, colour = 'blue')
viewer.add_isosurface(fault6[0], isovalue = 0, colour = 'deepblue')
viewer.add_isosurface(fault7[0], isovalue = 0, colour = 'lightblue')


# # add intrusion
viewer.add_isosurface(Northern_Sill, isovalue = 0, colour = 'red')
viewer.add_isosurface(Southern_Sill, isovalue = 0, colour = 'red')
viewer.ymin = 0
viewer.xmin = 0
viewer.zmin = 0
viewer.ymax = 1
viewer.xmax = 1
viewer.zmax = 1
viewer.interactive()

In [ ]:
datetime.now().isoformat(timespec='seconds')   